# Dataset


In [1]:
import pandas as pd

In [3]:
dataset_df = pd.read_csv(r"/content/Query.tsv",dtype=str,encoding='ISO-8859-1',sep='\t')

In [4]:
dataset_df

,Query
0,internet explorer
1,windowsmedia
2,microsoft
3,outlook.com
4,Preview window
...,...
244,meta
245,download internet explorer
246,encarta
247,windows live mail


In [5]:
dataset_df

,Query
0,internet explorer
1,windowsmedia
2,microsoft
3,outlook.com
4,Preview window
...,...
244,meta
245,download internet explorer
246,encarta
247,windows live mail


# Dataset preprocessing

In [6]:
%%capture
!pip install -U gensim

In [7]:
from gensim.utils import tokenize
from gensim.parsing.preprocessing import preprocess_string,strip_tags,strip_punctuation,strip_numeric,remove_stopwords,strip_short
from gensim.corpora.dictionary import Dictionary
from gensim import models

In [8]:
help(preprocess_string)

Help on function preprocess_string in module gensim.parsing.preprocessing:

preprocess_string(s, filters=[<function <lambda> at 0x7a33170879a0>, <function strip_tags at 0x7a3317087400>, <function strip_punctuation at 0x7a3317087370>, <function strip_multiple_whitespaces at 0x7a33170876d0>, <function strip_numeric at 0x7a33170875b0>, <function remove_stopwords at 0x7a3317087250>, <function strip_short at 0x7a3317087490>, <function stem_text at 0x7a33170877f0>])
    Apply list of chosen filters to `s`.
    
    Default list of filters:
    
    * :func:`~gensim.parsing.preprocessing.strip_tags`,
    * :func:`~gensim.parsing.preprocessing.strip_punctuation`,
    * :func:`~gensim.parsing.preprocessing.strip_multiple_whitespaces`,
    * :func:`~gensim.parsing.preprocessing.strip_numeric`,
    * :func:`~gensim.parsing.preprocessing.remove_stopwords`,
    * :func:`~gensim.parsing.preprocessing.strip_short`,
    * :func:`~gensim.parsing.preprocessing.stem_text`.
    
    Parameters
    -------

In [9]:
dataset_df['Clean_Queries'] = dataset_df['Query'].apply(preprocess_string)

In [10]:
dataset_df

,Query,Clean_Queries
0,internet explorer,"[internet, explor]"
1,windowsmedia,[windowsmedia]
2,microsoft,[microsoft]
3,outlook.com,"[outlook, com]"
4,Preview window,"[preview, window]"
...,...,...
244,meta,[meta]
245,download internet explorer,"[download, internet, explor]"
246,encarta,[encarta]
247,windows live mail,"[window, live, mail]"


In [11]:
filters=[lambda x: x.lower(),strip_tags,strip_punctuation,remove_stopwords,strip_short]
dataset_df['Clean_Queries1'] = dataset_df['Query'].apply(lambda x: preprocess_string(x,filters))

In [12]:
dataset_df

,Query,Clean_Queries,Clean_Queries1
0,internet explorer,"[internet, explor]","[internet, explorer]"
1,windowsmedia,[windowsmedia],[windowsmedia]
2,microsoft,[microsoft],[microsoft]
3,outlook.com,"[outlook, com]","[outlook, com]"
4,Preview window,"[preview, window]","[preview, window]"
...,...,...,...
244,meta,[meta],[meta]
245,download internet explorer,"[download, internet, explor]","[download, internet, explorer]"
246,encarta,[encarta],[encarta]
247,windows live mail,"[window, live, mail]","[windows, live, mail]"


In [13]:
dataset_dictionary = Dictionary(dataset_df['Clean_Queries1'])

In [14]:
len(dataset_dictionary)

201

In [15]:
print(dataset_dictionary.token2id)

{'explorer': 0, 'internet': 1, 'windowsmedia': 2, 'microsoft': 3, 'com': 4, 'outlook': 5, 'preview': 6, 'window': 7, 'homepage': 8, 'msn': 9, 'hotm': 10, 'skype': 11, 'account': 12, 'google': 13, 'toolbar': 14, 'history': 15, 'manage': 16, 'flash': 17, 'object': 18, 'ocx': 19, 'shockwave': 20, 'windows': 21, 'java': 22, 'office': 23, 'setup': 24, 'www': 25, 'media': 26, 'player': 27, 'update': 28, 'settings': 29, 'download': 30, 'free': 31, 'app': 32, 'store': 33, 'sharepoint': 34, 'marinenet': 35, 'mil': 36, 'usmc': 37, 'essentials': 38, 'security': 39, 'login': 40, 'downloads': 41, 'office365': 42, 'defender': 43, 'backgrounds': 44, 'desktop': 45, 'menu': 46, 'w2express': 47, 'support': 48, 'firefox': 49, 'mozilla': 50, 'live': 51, 'password': 52, 'reset': 53, 'background': 54, 'xbox': 55, 'maker': 56, 'movie': 57, 'help': 58, 'word': 59, 'mail': 60, '360': 61, 'email': 62, 'express': 63, 'proxy': 64, 'server': 65, 'onedrive': 66, 'device': 67, 'manager': 68, 'apk': 69, 'extension': 

In [16]:
dataset_corpus_bow = [dataset_dictionary.doc2bow(text) for text in dataset_df['Clean_Queries1']] #create a dataset corpus with bag of word vectorization

In [17]:
len(dataset_corpus_bow)

249

In [18]:
#Preview window
print(dataset_corpus_bow[4])

[(6, 1), (7, 1)]


In [19]:
tfidf = models.TfidfModel(dataset_corpus_bow)
dataset_corpus_tfidf = tfidf[dataset_corpus_bow]

In [20]:
len(dataset_corpus_tfidf)

249

In [21]:
print(dataset_corpus_tfidf[4])

[(6, 0.7071067811865475), (7, 0.7071067811865475)]


# Topic Modelling with Latent Dirichlet Allocation(LDA)


In [22]:
from gensim.models.ldamodel import LdaModel

In [23]:
lda_bow = LdaModel(dataset_corpus_bow,num_topics=20,id2word=dataset_dictionary,random_state=0)

In [24]:
lda_topics_bow = lda_bow.print_topics(num_words=8)
for topic in lda_topics_bow:
  print(topic)

(0, '0.101*"windows" + 0.076*"explorer" + 0.076*"internet" + 0.051*"toolbar" + 0.051*"mail" + 0.051*"outlook" + 0.051*"google" + 0.026*"flash"')
(1, '0.087*"windows" + 0.058*"extension" + 0.058*"file" + 0.058*"download" + 0.058*"desktop" + 0.030*"365" + 0.030*"2014" + 0.030*"chrome"')
(2, '0.071*"homepage" + 0.071*"xbox" + 0.071*"com" + 0.036*"setup" + 0.036*"firefox" + 0.036*"browser" + 0.036*"support" + 0.036*"explorer"')
(3, '0.044*"changer" + 0.044*"bing" + 0.044*"explorer" + 0.044*"daily" + 0.044*"browsing" + 0.044*"background" + 0.044*"desktop" + 0.044*"office"')
(4, '0.046*"updates" + 0.046*"internet" + 0.046*"download" + 0.046*"outlook" + 0.046*"access" + 0.046*"explorer" + 0.046*"java" + 0.046*"web"')
(5, '0.073*"email" + 0.073*"com" + 0.037*"ask" + 0.037*"account" + 0.037*"page" + 0.037*"inbox" + 0.037*"home" + 0.037*"open"')
(6, '0.152*"windows" + 0.048*"password" + 0.024*"change" + 0.024*"skype" + 0.024*"bing" + 0.024*"google" + 0.024*"w2express" + 0.024*"reset"')
(7, '0.08

In [25]:
lda_tfidf = LdaModel(dataset_corpus_tfidf, id2word=dataset_dictionary, num_topics=20)

In [26]:
lda_topics_tfidf = lda_tfidf.print_topics(num_words=8)
for topic in lda_topics_tfidf:
  print(topic)

(0, '0.056*"log" + 0.053*"account" + 0.043*"app" + 0.043*"iexplorer" + 0.043*"theme" + 0.043*"java" + 0.043*"miracast" + 0.034*"menu"')
(1, '0.075*"update" + 0.045*"windows" + 0.045*"download" + 0.034*"desktop" + 0.032*"explorer" + 0.032*"internet" + 0.030*"bing" + 0.029*"w2express"')
(2, '0.052*"desktop" + 0.052*"net" + 0.046*"downloads" + 0.043*"microsoft" + 0.040*"speakers" + 0.039*"help" + 0.035*"inbox" + 0.035*"bluetooth"')
(3, '0.055*"msn" + 0.055*"homepage" + 0.041*"drive" + 0.033*"antivirus" + 0.032*"acsm" + 0.031*"ssv" + 0.031*"email" + 0.025*"2014"')
(4, '0.054*"zune" + 0.054*"hotmil" + 0.054*"settings" + 0.054*"mse" + 0.054*"msconfig" + 0.054*"internet" + 0.039*"app" + 0.039*"security"')
(5, '0.067*"java" + 0.061*"history" + 0.057*"download" + 0.048*"wallpaper" + 0.048*"nslookup" + 0.048*"cookies" + 0.041*"directx" + 0.039*"delete"')
(6, '0.063*"msn" + 0.051*"xbox" + 0.045*"download" + 0.042*"microsoft" + 0.041*"account" + 0.031*"onedrive" + 0.031*"trovi" + 0.027*"music"')
(

# Topic Modelling with Latent Semantic Analysis/Indexing(LSA/LSI)



In [27]:
from gensim.models.lsimodel import LsiModel

In [28]:
lsi_bow = LsiModel(corpus=dataset_corpus_bow,id2word=dataset_dictionary,num_topics=20)

In [29]:
lsi_topics_bow = lsi_bow.print_topics(num_words=8)
for topic in lsi_topics_bow:
  print(topic)

(0, '0.760*"com" + 0.366*"www" + 0.353*"microsoft" + 0.175*"windows" + 0.156*"outlook" + 0.135*"office" + 0.110*"java" + 0.108*"download"')
(1, '-0.886*"windows" + -0.204*"download" + 0.184*"com" + -0.168*"internet" + -0.168*"explorer" + -0.124*"update" + 0.088*"www" + -0.081*"free"')
(2, '0.650*"internet" + 0.649*"explorer" + -0.293*"windows" + 0.194*"download" + 0.096*"update" + 0.061*"homepage" + 0.052*"adblock" + 0.052*"browsing"')
(3, '-0.861*"microsoft" + 0.313*"com" + -0.165*"login" + 0.160*"www" + 0.135*"outlook" + -0.114*"365" + -0.095*"essentials" + -0.088*"downloads"')
(4, '-0.690*"extension" + -0.690*"file" + -0.082*"exe" + -0.077*"rtc" + -0.077*"lnk" + -0.077*"dmp" + -0.077*"acsm" + -0.077*"emz"')
(5, '-0.698*"download" + 0.312*"live" + -0.292*"skype" + -0.259*"java" + -0.201*"free" + 0.163*"xbox" + 0.145*"internet" + 0.144*"explorer"')
(6, '0.544*"live" + 0.541*"xbox" + 0.296*"download" + 0.283*"free" + 0.187*"games" + 0.187*"latest" + -0.169*"www" + 0.166*"account"')
(7,

In [30]:
lsi_tfidf = LsiModel(dataset_corpus_tfidf, id2word=dataset_dictionary, num_topics=20)

In [31]:
lsi_topics_tfidf = lsi_tfidf.print_topics(num_words=8)
for topic in lsi_topics_tfidf:
  print(topic)

(0, '-0.614*"explorer" + -0.565*"internet" + -0.367*"windows" + -0.267*"download" + -0.207*"update" + -0.093*"com" + -0.086*"java" + -0.082*"skype"')
(1, '-0.546*"microsoft" + -0.495*"com" + -0.309*"www" + 0.233*"explorer" + 0.211*"internet" + -0.204*"windows" + -0.177*"outlook" + -0.152*"java"')
(2, '-0.815*"windows" + 0.270*"explorer" + 0.242*"internet" + 0.237*"microsoft" + 0.189*"com" + -0.130*"update" + 0.125*"www" + -0.095*"live"')
(3, '-0.655*"microsoft" + 0.384*"com" + 0.271*"outlook" + 0.264*"java" + 0.263*"www" + 0.180*"skype" + -0.154*"essentials" + -0.152*"login"')
(4, '-0.581*"xbox" + -0.520*"live" + 0.327*"download" + 0.318*"skype" + 0.232*"java" + -0.141*"mail" + -0.121*"account" + 0.115*"www"')
(5, '0.503*"skype" + 0.469*"download" + -0.425*"update" + 0.339*"xbox" + -0.222*"outlook" + 0.193*"free" + 0.149*"live" + -0.148*"java"')
(6, '0.672*"extension" + 0.672*"file" + 0.122*"exe" + 0.111*"rtc" + 0.109*"acsm" + 0.109*"deskthemepack" + 0.109*"apk" + 0.109*"lnk"')
(7, '-0

# Topic Modelling Visualization with pyLDAvis

In [32]:
%%capture
!pip install pyLDAvis

In [33]:
import pyLDAvis
import pyLDAvis.gensim_models

In [34]:
pyLDAvis.enable_notebook()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
vis_bow = pyLDAvis.gensim_models.prepare(lda_bow, dataset_corpus_bow, dataset_dictionary)
vis_bow

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11    -0.108282 -0.119594       1        1  9.974736
12     0.012320 -0.142357       2        1  7.969372
0      0.161705 -0.055804       3        1  7.932313
18     0.054249 -0.046985       4        1  7.196289
17     0.050614 -0.038837       5        1  6.607920
6      0.096163  0.117244       6        1  6.298319
14     0.111268  0.080608       7        1  5.451354
8     -0.140305 -0.038445       8        1  5.100690
15    -0.116376  0.178620       9        1  4.971590
1      0.136943 -0.031105      10        1  4.914340
5     -0.129044  0.001923      11        1  4.322866
7     -0.134270  0.045158      12        1  4.206610
3     -0.021597  0.031664      13        1  3.994774
16     0.033276  0.082353      14        1  3.273787
19     0.086088  0.011517      15        1  3.194151
9     -0.069474 -0.021735      16        1  3.194150
13     0.002273  0.109407      17        1  3.099373
2      0.021717 -0.061416      18        1  3.077285
4      0.042620 -0.034702      19        1  2.773593
10    -0.089888 -0.067516      20        1  2.446490, topic_info=         Term       Freq      Total Category  logprob  loglift
21    windows  25.000000  25.000000  Default  30.0000  30.0000
3   microsoft  16.000000  16.000000  Default  29.0000  29.0000
4         com  22.000000  22.000000  Default  28.0000  28.0000
30   download  10.000000  10.000000  Default  27.0000  27.0000
55       xbox   6.000000   6.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
13     google   0.024195   4.949767  Topic20  -6.2934  -1.6105
15    history   0.024195   3.319908  Topic20  -6.2934  -1.2110
23     office   0.024195   6.692070  Topic20  -6.2934  -1.9120
25        www   0.024195  10.029101  Topic20  -6.2934  -2.3166
29   settings   0.024195   2.791197  Topic20  -6.2934  -1.0376

[824 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
162       2  0.732367    2013
192      10  0.675798    2014
61       15  0.770247     360
79        1  0.211199     365
79        3  0.211199     365
...     ...       ...     ...
55        7  0.460433    xbox
55       15  0.153478    xbox
55       18  0.153478    xbox
143      12  0.614248  xboxie
156       7  0.778525    zune

[402 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 13, 1, 19, 18, 7, 15, 9, 16, 2, 6, 8, 4, 17, 20, 10, 14, 3, 5, 11])

In [ ]:
vis_tfidf = pyLDAvis.gensim_models.prepare(lda_tfidf, dataset_corpus_tfidf, dataset_dictionary)
vis_tfidf

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
14    -0.025952  0.002916       1        1  6.825769
7     -0.084587  0.013373       2        1  6.420266
1      0.044758  0.153243       3        1  6.080071
19     0.120393  0.034161       4        1  5.801025
9     -0.029249 -0.005755       5        1  5.717830
18     0.020307  0.050408       6        1  5.686163
8      0.091035 -0.034451       7        1  5.579452
16    -0.117450  0.123358       8        1  5.555034
6      0.096050  0.072669       9        1  5.438963
17    -0.048143 -0.094249      10        1  5.337110
13     0.013750 -0.000265      11        1  5.334844
2      0.051215 -0.016363      12        1  5.264664
10     0.039833 -0.073863      13        1  5.125934
0     -0.141885  0.049740      14        1  4.533473
15     0.039228 -0.090904      15        1  3.892192
11    -0.049270 -0.042602      16        1  3.792407
12    -0.117253 -0.086075      17        1  3.611728
5      0.052135 -0.059529      18        1  3.448036
3      0.052517  0.008354      19        1  3.390595
4     -0.007431 -0.004167      20        1  3.164445, topic_info=         Term      Freq     Total Category  logprob  loglift
21    windows  9.000000  9.000000  Default  30.0000  30.0000
3   microsoft  7.000000  7.000000  Default  29.0000  29.0000
4         com  7.000000  7.000000  Default  28.0000  28.0000
22       java  4.000000  4.000000  Default  27.0000  27.0000
51       live  4.000000  4.000000  Default  26.0000  26.0000
..        ...       ...       ...      ...      ...      ...
22       java  0.031194  4.011421  Topic20  -5.8572  -1.4035
23     office  0.031194  3.513269  Topic20  -5.8572  -1.2709
24      setup  0.031194  2.119365  Topic20  -5.8572  -0.7654
25        www  0.031194  4.727319  Topic20  -5.8572  -1.5677
29   settings  0.031194  2.241479  Topic20  -5.8572  -0.8215

[816 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
162       8  0.880724     2013
79        9  0.418783      365
12        9  0.336714  account
12       11  0.336714  account
149       1  0.894741     acsm
...     ...       ...      ...
55       10  0.215699     xbox
55       17  0.215699     xbox
55       18  0.215699     xbox
143       6  0.778924   xboxie
156      18  0.805815     zune

[196 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 8, 2, 20, 10, 19, 9, 17, 7, 18, 14, 3, 11, 1, 16, 12, 13, 6, 4, 5])

# Model evaluation for Topic Modelling

In [36]:
from gensim.models import CoherenceModel

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
texts= dataset_df['Clean_Queries1']
texts = [x for x in texts if x]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
cm_lda_bow_cv = CoherenceModel(model=lda_bow,texts=texts,dictionary=dataset_dictionary,coherence='c_v')
cm_lda_bow_cv.get_coherence()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.6722894548901733

In [39]:
cm_lsi_bow_cv = CoherenceModel(model=lsi_bow, texts=texts, dictionary=dataset_dictionary, coherence='c_v')
cm_lsi_bow_cv.get_coherence()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.6387972651957761